# Iterative pruning pipeline
Model: Multi Layer Perceptron

*Pruning functions as class methods*

In [4]:
EXPERIMENT_NAME = 'mlp-global-magnitude-unstruct'
ITERATIONS = 3 #should be 10 for final experiment


In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import json
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import foolbox as fb
import random


tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Prune, Train Attack Pipeline

In [ ]:

pgd_success_rates = []
cw2_success_rates = []

all_accuracies = []
for j in tqdm(range(ITERATIONS)):
    model = initialize_base_model(j, save_weights=True)

    accuracies = []
    pgd_success_rate = []
    cw2_success_rate = []
    compression_rates = [1, 2, 4, 8, 16, 32, 64]
    pruning_ratios = [1-1/x for x in compression_rates]
    for index, pruning_ratio in tqdm(enumerate(pruning_ratios)):
        model.load_weights(f'./saved-weights/{EXPERIMENT_NAME}-{j}')

        #iteratively prune and train (only to convergence if the final stage of pruning is reached)
        for i in range(index + 1):
            if i != index:
                #glocbal pruning
                model.prune_random_global_unstruct(pruning_ratio)
                #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                #              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                #              metrics=['accuracy'],
                #             )
                             
                #finetuning
                model = train_model(model, to_convergence=False)
            if i == index:
                print('final pruning and eval')
                model.prune_random_global_unstruct(pruning_ratio)
                #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                #              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                #              metrics=['accuracy'],
                #             )
                #finetuning
                model = train_model(model, to_convergence=True)
                accuracies.append(model.evaluate(x_test, y_test, verbose=0))
                pgd_success_rate.append(pgd_attack(model))
                #cw2_success_rate.append(cw2_attack(model))
    all_accuracies.append(accuracies)
    pgd_success_rates.append(pgd_success_rate)
    cw2_success_rates.append(cw2_success_rate)

    
pd.DataFrame(all_accuracies).to_csv(f'saved-results/{EXPERIMENT_NAME}-accuracies.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-accuracies.json', 'w') as f:
    json.dump(all_accuracies, f)
    
pd.DataFrame(pgd_success_rates).to_csv(f'saved-results/{EXPERIMENT_NAME}-pgd-success.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-pgd-success.json', 'w') as f:
    json.dump(pgd_success_rates, f)
    
pd.DataFrame(cw2_success_rates).to_csv(f'saved-results/{EXPERIMENT_NAME}-cw2-success.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-cw2-success.json', 'w') as f:
    json.dump(cw2_success_rates, f)

  0%|          | 0/3 [00:00<?, ?it/s]

938/938 [==============================] - 6s 6ms/step - loss: 1.6108 - accuracy: 0.8625 - val_loss: 1.5174 - val_accuracy: 0.9477



0it [00:00, ?it/s]

final pruning and eval
Epoch 1/100
938/938 [==============================] - 5s 6ms/step - loss: 1.5105 - accuracy: 0.9532 - val_loss: 1.5050 - val_accuracy: 0.9585
Epoch 2/100
938/938 [==============================] - 5s 6ms/step - loss: 1.4975 - accuracy: 0.9655 - val_loss: 1.5037 - val_accuracy: 0.9588
Epoch 3/100
938/938 [==============================] - 5s 6ms/step - loss: 1.4903 - accuracy: 0.9722 - val_loss: 1.4926 - val_accuracy: 0.9690
Epoch 4/100
938/938 [==============================] - 5s 6ms/step - loss: 1.4865 - accuracy: 0.9757 - val_loss: 1.4911 - val_accuracy: 0.9709
Epoch 5/100
938/938 [==============================] - 5s 6ms/step - loss: 1.4825 - accuracy: 0.9794 - val_loss: 1.4894 - val_accuracy: 0.9715
Epoch 6/100
938/938 [==============================] - 5s 6ms/step - loss: 1.4804 - accuracy: 0.9817 - val_loss: 1.4898 - val_accuracy: 0.9721
Epoch 7/100
938/938 [==============================] - 5s 6ms/step - loss: 1.4791 - accuracy: 0.9827 - val_loss: 1.4857


1it [01:14, 74.87s/it]

...

Epoch 1/2
938/938 [==============================] - 5s 6ms/step - loss: 1.7204 - accuracy: 0.7872 - val_loss: 1.5514 - val_accuracy: 0.9222
Epoch 2/2
938/938 [==============================] - 6s 6ms/step - loss: 1.5448 - accuracy: 0.9254 - val_loss: 1.5354 - val_accuracy: 0.9327
Epoch 1/2
938/938 [==============================] - 6s 7ms/step - loss: 1.5290 - accuracy: 0.9383 - val_loss: 1.5246 - val_accuracy: 0.9412
Epoch 2/2
938/938 [==============================] - 5s 5ms/step - loss: 1.5188 - accuracy: 0.9472 - val_loss: 1.5202 - val_accuracy: 0.9442
Epoch 1/2
938/938 [==============================] - 5s 6ms/step - loss: 1.5117 - accuracy: 0.9536 - val_loss: 1.5142 - val_accuracy: 0.9499
Epoch 2/2
938/938 [==============================] - 5s 6ms/step - loss: 1.5070 - accuracy: 0.9580 - val_loss: 1.5105 - val_accuracy: 0.9527
final pruning and eval
Epoch 1/100
938/938 [==============================] - 5s 5ms/step - loss: 1.5028 - accuracy: 0.9620 - val_loss: 1.5072 - val_accur


4it [07:13, 106.56s/it]

Epoch 1/2
938/938 [==============================] - 5s 5ms/step - loss: 1.9638 - accuracy: 0.5416 - val_loss: 1.5879 - val_accuracy: 0.8990
Epoch 2/2
938/938 [==============================] - 5s 5ms/step - loss: 1.5717 - accuracy: 0.9054 - val_loss: 1.5539 - val_accuracy: 0.9168
Epoch 1/2
938/938 [==============================] - 5s 6ms/step - loss: 1.5507 - accuracy: 0.9193 - val_loss: 1.5439 - val_accuracy: 0.9235
Epoch 2/2
938/938 [==============================] - 5s 5ms/step - loss: 1.5409 - accuracy: 0.9279 - val_loss: 1.5366 - val_accuracy: 0.9292
Epoch 1/2
938/938 [==============================] - 5s 5ms/step - loss: 1.5340 - accuracy: 0.9329 - val_loss: 1.5323 - val_accuracy: 0.9328
Epoch 2/2
938/938 [==============================] - 5s 5ms/step - loss: 1.5289 - accuracy: 0.9374 - val_loss: 1.5284 - val_accuracy: 0.9364
Epoch 1/2
938/938 [==============================] - 5s 5ms/step - loss: 1.5247 - accuracy: 0.9415 - val_loss: 1.5260 - val_accuracy: 0.9393
Epoch 2/2
938


5it [11:18, 148.20s/it]

Epoch 1/2
938/938 [==============================] - 6s 6ms/step - loss: 2.2565 - accuracy: 0.1646 - val_loss: 2.2032 - val_accuracy: 0.2654
Epoch 2/2
938/938 [==============================] - 5s 6ms/step - loss: 2.0491 - accuracy: 0.3997 - val_loss: 1.9404 - val_accuracy: 0.4866
Epoch 1/2
938/938 [==============================] - 6s 6ms/step - loss: 1.8887 - accuracy: 0.5200 - val_loss: 1.8623 - val_accuracy: 0.5353
Epoch 2/2
938/938 [==============================] - 6s 6ms/step - loss: 1.8490 - accuracy: 0.5490 - val_loss: 1.8178 - val_accuracy: 0.5775
Epoch 1/2
938/938 [==============================] - 5s 6ms/step - loss: 1.8095 - accuracy: 0.5864 - val_loss: 1.7976 - val_accuracy: 0.5973
Epoch 2/2
938/938 [==============================] - 6s 6ms/step - loss: 1.7953 - accuracy: 0.5989 - val_loss: 1.7869 - val_accuracy: 0.6057
Epoch 1/2
938/938 [==============================] - 5s 6ms/step - loss: 1.7864 - accuracy: 0.6073 - val_loss: 1.7802 - val_accuracy: 0.6100
Epoch 2/2
938

# Helper Functions

In [6]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=2,
            validation_data=(x_test, y_test),
            )
    return model



def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 0) or (index == 2) or (index == 4):
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index].flatten()
            mask = weights_to_prune[index+1].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            #mask_df = pd.DataFrame(mask)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            #print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                mask[idx_to_delete] = 0
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+1].shape
            mask_reshaped = mask.reshape(dims)
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            weights_to_prune[index+1] = mask_reshaped
            weights_to_prune[index] = weights_reshaped
    
    return weights_to_prune



def pgd_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.LinfProjectedGradientDescentAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[15/255]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def cw2_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.L2CarliniWagnerAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[.5]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def initialize_base_model(index, save_weights=False):
    model = LeNet300_100()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=x_train,
              y=y_train,
              batch_size=64,
              epochs=1,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )
    if save_weights == True:
        model.save_weights(f'./saved-weights/{EXPERIMENT_NAME}-{index}')
    return model
    

# Load Data

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x = tf.convert_to_tensor(x_train[:500].reshape(500,28*28))
y = tf.convert_to_tensor([y_train[:500]])[0];

# Define Model

In [8]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                name='unpruned_weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning_mask')

        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        x = tf.multiply(self.w, self.mask)
        #print(self.pruned_w.eval())
        x = tf.matmul(inputs, x)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')

class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
    def prune_random_global_unstruct(self, ratio):
        weights = self.get_weights()
        layers_to_prune = [0, 2, 4]
        flat_weights = np.array([])
        flat_mask = np.array([])
        for x in layers_to_prune:
            flat_weights = np.concatenate([flat_weights, weights[x].flatten()])
            flat_mask = np.concatenate([flat_mask, weights[x+1].flatten()])
        no_of_weights_to_prune = ratio * len(flat_weights)
        # find unpruned weights
        non_zero_weights = np.nonzero(flat_mask)[0]
        # calculate the amount of weights to be pruned this round
        no_of_weights_to_prune_left = int(no_of_weights_to_prune - (len(flat_weights) - len(non_zero_weights)) )
        # shuffle all non-zero weights
        random.shuffle(non_zero_weights)
        # and take the indices of the first x weights where x is the number of weights to be pruned this round
        indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]
        for idx_to_delete in indices_to_delete:
            flat_mask[idx_to_delete] = 0
            flat_weights[idx_to_delete] = 0
            
        #reshape
        z = 0
        for x in layers_to_prune:
            weights[x] = flat_weights[z:z + np.prod(weights[x].shape)].reshape(weights[x].shape)
            weights[x + 1] = flat_mask[z:z + np.prod(weights[x].shape)].reshape(weights[x].shape)
            z = z + np.prod(weights[x].shape)
        self.set_weights(weights)
        return True
        
            
        
        
    def prune_random_local_unstruct(self, ratio):
        layers = self.get_weights()
        layers_to_prune = [0, 2, 4]
        for index, weights in enumerate(layers):
            if index in layers_to_prune:
                shape = weights.shape
                flat_weights = weights.flatten()
                flat_mask = layers[index+1].flatten()
                no_of_weights_to_prune = int(len(flat_weights)*ratio)
                # find unpruned weights
                non_zero_weights = np.nonzero(flat_mask)[0]
                # calculate the amount of weights to be pruned this round
                no_of_weights_to_prune_left = int(no_of_weights_to_prune - (len(flat_weights) - len(non_zero_weights)) )
                # shuffle all non-zero weights
                random.shuffle(non_zero_weights)
                # and take the indices of the first x weights where x is the number of weights to be pruned this round
                indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]
                for idx_to_delete in indices_to_delete:
                    flat_mask[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                
                mask_reshaped = flat_mask.reshape(shape)
                weights_reshaped = flat_weights.reshape(shape)
                layers[index+1] = mask_reshaped
                layers[index] = weights_reshaped
        self.set_weights(layers)
        return True
        
    
    def prune_magnitude_global_unstruct(self,ratio):
                
        shape1 = self.dense1.w.shape
        shape2 = self.dense2.w.shape
        shape3 = self.dense3.w.shape

        flat_weights = np.append(self.dense1.w.numpy().flatten() ,self.dense2.w.numpy().flatten())
        flat_weights = np.append(flat_weights ,self.dense3.w.numpy().flatten())
        flat_mask = np.append(self.dense1.mask.numpy().flatten(), self.dense2.mask.numpy().flatten())
        flat_mask = np.append(flat_mask, self.dense3.mask.numpy().flatten())
        
        no_of_weights_to_prune = int(len(flat_weights)*ratio)
        indices_to_delete = np.abs(flat_weights).argsort()[:no_of_weights_to_prune]
        
        for idx_to_delete in indices_to_delete:
            flat_mask[idx_to_delete] = 0
            flat_weights[idx_to_delete] = 0
            
        w1 = flat_weights[:shape1[0]*shape1[1]].reshape(shape1)
        w2 = flat_weights[shape1[0]*shape1[1]:shape1[0]*shape1[1]+shape2[0]*shape2[1]].reshape(shape2)
        w3 = flat_weights[-shape3[0]*shape3[1]:].reshape(shape3)
        m1 = flat_mask[:shape1[0]*shape1[1]].reshape(shape1)
        m2 = flat_mask[shape1[0]*shape1[1]:shape1[0]*shape1[1]+shape2[0]*shape2[1]].reshape(shape2)
        m3 = flat_mask[-shape3[0]*shape3[1]:].reshape(shape3)
        self.set_weights([w1,m1,w2,m2,w3,m3])
        #print(weights)
        return True
    
    def prune_magnitude_local_unstruct(self, ratio):
        layers = self.get_weights()
        layers_to_prune = [0, 2, 4]
        for index, weights in enumerate(layers):
            if index in layers_to_prune:
                shape = weights.shape
                flat_weights = weights.flatten()
                mask = layers[index+1].flatten()
                
                no_of_weights_to_prune = int(len(flat_weights)*ratio)
                indices_to_delete = np.abs(flat_weights).argsort()[:no_of_weights_to_prune]
                for idx_to_delete in indices_to_delete:
                    mask[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                
                mask_reshaped = mask.reshape(shape)
                weights_reshaped = flat_weights.reshape(shape)
                layers[index+1] = mask_reshaped
                layers[index] = weights_reshaped
        self.set_weights(layers)
        return True

# Compile and Train Model

In [ ]:
model = LeNet300_100()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=1,
          callbacks=[callback],
          validation_data=(x_test, y_test),
         )

model.save('./saved-models/mini-pipeline-mlp-baseline-model')
model.save_weights('./saved-models/weights')

In [ ]:
weights = np.array(list(range(100)));zz

In [ ]:

no_of_weighs_to_prune = rate * len(weights)

non_zero_weights = np.nonzero(zz)[0]
no_of_weights_to_prune_left = int(no_of_weighs_to_prune - (len(weights) - len(non_zero_weights)) )

random.shuffle(non_zero_weights)
indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]